In [1]:
import csv
import pandas as pd
import numpy as np
import datetime
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.impute import SimpleImputer

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

data = []

data_1 = 'RESIDIV_Vimala.csv'
data_2 = 'VV_RESIDIV_Vimala.csv'

In [2]:
data_labeled = pd.read_csv(data_1)
data_unlabeled = pd.read_csv(data_2) 

In [3]:
data_unlabeled[:20]

,Pas_ID,Født,Oppdaget_når,Age,Død,lever,FIGO_stadium,Substadium,histologi,gradering,kreftform,OVAgrade,myometireinvasjon,Cx_invasjon,Karinvasjon,Lymfekarinvasjon,Lymfeknuter,Tumor_str,hist_nummer,Primærbehandling,Avsluttet_prim_beh,Ant_ktr,Beh_sykehus,Ktr_ingen,Ktr_priv,Ktr_lok,Ktr_Uni,Ktr_komb,Ktr_fastlege,Ktr_andre,Lengde_sympt_dager,Lengde_sympt_uker,Lengde_sympt_mnd,Symptomer_mindre_1mnd,TidSympt_2dic,Oppdaget_pas,Oppdaget_FL,Oppdaget_Priv,Oppdaget_Lok,Oppdaget_Univ,Oppdaget_Kir,Oppdaget_annen,Tidligere_kontakt,Sympt_tilstede,Sympt_blødning,Sympt_smerter,Sympt_vekttap,Sympt_utflod,Sympt_ascites,Sympt_avføring,Sympt_ileus,Sympt_fatigue,Sympt_vannlating,Sympt_blodfeces,Sympt_blodurin,Sympt_anorexi,Sympt_kakeksi,Sympt_hudtumor,sympt_luftveier,Sympt_annet,Verif_histo,Histo_nummer,Histo_lab,Verif_cytol,Cytol_nummer,Cytol_lab,Verif_MR,Verif_CT,Verif_RTH,Verif_ABD_UL,Verif_VAG_UL,ULverif,Verif_Colon,Verif_annet,Lok_vagina,Lok_bekken,Lok_ØvreAbd,Lok_lunger,Lok_lever,Lok_lok_LK,Lok_fjern_LK,Lok_skjelett,Lok_hjernen,Lok_annet,Henvist_Uni,Hvorfor_ikke,Behandling,Time_to_recurrence_months,symptomer_kliniske,KlinSympt_2dic,symptomer,Symtomer_tilstede,Sympt_tilstede2dic,Symptomer_lengde,Distal_recurrence,Local_recurrence,Age_at_diagnosis,age_rec
0,5013,3/22/1937,1/1/2013,75,,,1,3,1,2,1,,,2,2,,2,,,1,3/1/2009,8,5,0,0,0,5,0,0,,5,0,0,1,1,1,0,0,0,0,0,,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,,0,,,1,4156,5,0,1,0,1,1,1,0,,0,0,0,0,0,0,1,0,0,,2,konsultert HUS,2,46,1,1,5,2,1,0,1,0,71,75
1,1244,3/4/1941,12/1/2015,74,,1,1,2,1,2,1,,2,2,1,,,,1503931,1,3/1/2015,3,,0,0,0,0,,0,,0,1,0,1,1,1,0,0,1,1,0,,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,,1,15160,1,1,16000088,,0,1,0,1,1,1,0,,1,0,1,0,0,0,1,0,0,,1,,3,9,3,1,1,2,1,0,1,0,73,74
2,2116,4/27/1934,8/1/2014,80,,,1,3,1,1,1,,,,,,,,819216,5,1/1/2009,16,2,0,1,0,0,0,0,,0,0,1,0,1,1,0,1,0,1,0,,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,,1,1427589,2,0,,,0,1,0,1,1,1,0,,0,1,0,0,0,0,0,0,0,,1,,3,67,1,1,1,2,1,2,0,1,74,80
3,5017,5/21/1935,11/1/2013,78,,,1,1,6,,1,,,,,,,,4884,1,3/1/2013,2,5,0,0,0,5,0,0,,0,3,0,1,1,0,0,1,0,0,0,,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,,1,632,5,0,,,0,1,0,1,1,1,0,,0,1,1,0,0,0,0,0,0,,1,,3,8,1,1,3,2,1,0,1,0,77,78
4,5039,2/15/1936,8/1/2015,79,,,3,3,6,,1,,,,,,,,23874,5,4/1/2015,1,5,0,0,0,5,0,0,,0,0,0,0,0,0,0,0,0,1,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,1,17694,5,0,,,0,1,0,1,1,1,0,,1,0,0,0,0,0,0,0,0,,1,,3,4,0,0,0,0,0,1,0,1,79,79
5,11012,4/20/1932,11/1/2015,83,,1,1,2,1,2,1,,2,2,1,,,,12628,1,7/1/2014,6,11,0,1,0,0,0,0,,0,0,0,0,0,0,0,1,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,0,,,0,,,1,0,0,0,1,1,0,,0,1,0,0,0,0,0,0,0,,1,,,16,0,0,0,0,0,1,0,1,82,83
6,29014,8/26/1931,2/1/2014,82,,1,1,2,1,2,1,,2,2,2,3,2,,,5,11/1/2011,7,29,0,0,1,0,0,0,,0,0,0,0,0,0,0,0,1,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,0,,,1,1643,16,1,0,0,0,1,1,0,,1,0,0,0,0,0,0,0,0,,,,1,27,0,0,0,0,0,1,0,1,80,82
7,1150,4/22/1936,5/1/2014,78,,,1,2,1,1,1,,2,2,2,3,2,,1307185,1,8/1/2013,3,1,0,0,1,0,0,0,,0,0,0,0,0,0,0,0,1,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,1,12883,8,0,,,0,1,0,0,1,1,0,,1,0,0,0,0,0,0,0,0,,1,,2,9,0,0,0,0,0,1,0,1,77,78
8,1176,4/27/1934,11/1/2014,80,,,1,2,1,2,1,,2,2,1,3,3,,,1,4/1/2014,2,6,0,0,0,6,0,0,,0,0,12,0,1,1,0,0,1,0,0,,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,,1,13266,6,0,,,0,1,1,0,1,1,0,,1,0,0,0,0,0,0,0,0,,1,,2,7,1,1,12,2,1,13,0,1,79,80
9,1184,3/18/1927,12/1/2014,87,,,1,2,12,1,1,,2,2,1,,,,1111414,1,2/1/2011,10,31,0,0,1,0,0,0,,0,0,0,0,0,0,0,0,1,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,1,1500651,,0,,,0,1,0,0,1,1,0,,0,0,0,0,0,1,0,0,0,,1,,2,46,0,0,0,0,0,1,1,0,83,87


In [4]:
raw_data = data_unlabeled

In [5]:
def convert_time_to_integer(value):
    r = re.compile('\d*\/\d*\/\d*')
    if r.match(value) == None:
        pass
    else:
        month, date, year = value.split('/')
        date = pd.to_numeric(date, errors='coerce')
        month = pd.to_numeric(month, errors='coerce')
        year = pd.to_numeric(year, errors='coerce')
    
        return datetime.datetime(year, month, date).timestamp()/10**6

In [6]:
data_unlabeled['Født'] = data_unlabeled['Født'].apply(convert_time_to_integer)
data_unlabeled['Død'] = data_unlabeled['Død'].apply(convert_time_to_integer)
data_unlabeled['Oppdaget_når'] = data_unlabeled['Oppdaget_når'].apply(convert_time_to_integer)
data_unlabeled['Avsluttet_prim_beh'] = data_unlabeled['Avsluttet_prim_beh'].apply(convert_time_to_integer)


def fill_nan_with_mean(df):
    return df.fillna(df.mean())

data_unlabeled['Født'] = fill_nan_with_mean(data_unlabeled['Født'])
data_unlabeled['Oppdaget_når'] = fill_nan_with_mean(data_unlabeled['Oppdaget_når'])
data_unlabeled['Avsluttet_prim_beh'] = fill_nan_with_mean(data_unlabeled['Avsluttet_prim_beh'])


In [7]:
data_unlabeled.head()

,Pas_ID,Født,Oppdaget_når,Age,Død,lever,FIGO_stadium,Substadium,histologi,gradering,kreftform,OVAgrade,myometireinvasjon,Cx_invasjon,Karinvasjon,Lymfekarinvasjon,Lymfeknuter,Tumor_str,hist_nummer,Primærbehandling,Avsluttet_prim_beh,Ant_ktr,Beh_sykehus,Ktr_ingen,Ktr_priv,Ktr_lok,Ktr_Uni,Ktr_komb,Ktr_fastlege,Ktr_andre,Lengde_sympt_dager,Lengde_sympt_uker,Lengde_sympt_mnd,Symptomer_mindre_1mnd,TidSympt_2dic,Oppdaget_pas,Oppdaget_FL,Oppdaget_Priv,Oppdaget_Lok,Oppdaget_Univ,Oppdaget_Kir,Oppdaget_annen,Tidligere_kontakt,Sympt_tilstede,Sympt_blødning,Sympt_smerter,Sympt_vekttap,Sympt_utflod,Sympt_ascites,Sympt_avføring,Sympt_ileus,Sympt_fatigue,Sympt_vannlating,Sympt_blodfeces,Sympt_blodurin,Sympt_anorexi,Sympt_kakeksi,Sympt_hudtumor,sympt_luftveier,Sympt_annet,Verif_histo,Histo_nummer,Histo_lab,Verif_cytol,Cytol_nummer,Cytol_lab,Verif_MR,Verif_CT,Verif_RTH,Verif_ABD_UL,Verif_VAG_UL,ULverif,Verif_Colon,Verif_annet,Lok_vagina,Lok_bekken,Lok_ØvreAbd,Lok_lunger,Lok_lever,Lok_lok_LK,Lok_fjern_LK,Lok_skjelett,Lok_hjernen,Lok_annet,Henvist_Uni,Hvorfor_ikke,Behandling,Time_to_recurrence_months,symptomer_kliniske,KlinSympt_2dic,symptomer,Symtomer_tilstede,Sympt_tilstede2dic,Symptomer_lengde,Distal_recurrence,Local_recurrence,Age_at_diagnosis,age_rec
0,5013,-1034.4708,1356.9948,75,NaN,,1,3,1,2,1,,,2,2,,2,,,1,1235.8620,8,5,0,0,0,5,0,0,,5,0,0,1,1,1,0,0,0,0,0,,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,,0,,,1,4156,5,0,1,0,1,1,1,0,,0,0,0,0,0,0,1,0,0,,2,konsultert HUS,2,46,1,1,5,2,1,0,1,0,71,75
1,1244,-909.7992,1448.9244,74,NaN,1,1,2,1,2,1,,2,2,1,,,,1503931,1,1425.1644,3,,0,0,0,0,,0,,0,1,0,1,1,1,0,0,1,1,0,,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,,1,15160,1,1,16000088,,0,1,0,1,1,1,0,,1,0,1,0,0,0,1,0,0,,1,,3,9,3,1,1,2,1,0,1,0,73,74
2,2116,-1126.0548,1406.8440,80,NaN,,1,3,1,1,1,,,,,,,,819216,5,1230.7644,16,2,0,1,0,0,0,0,,0,0,1,0,1,1,0,1,0,1,0,,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,,1,1427589,2,0,,,0,1,0,1,1,1,0,,0,1,0,0,0,0,0,0,0,,1,,3,67,1,1,1,2,1,2,0,1,74,80
3,5017,-1092.4452,1383.2604,78,NaN,,1,1,6,,1,,,,,,,,4884,1,1362.0924,2,5,0,0,0,5,0,0,,0,3,0,1,1,0,0,1,0,0,0,,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,,1,632,5,0,,,0,1,0,1,1,1,0,,0,1,1,0,0,0,0,0,0,,1,,3,8,1,1,3,2,1,0,1,0,77,78
4,5039,-1069.1172,1438.3800,79,NaN,,3,3,6,,1,,,,,,,,23874,5,1427.8392,1,5,0,0,0,5,0,0,,0,0,0,0,0,0,0,0,0,1,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,1,17694,5,0,,,0,1,0,1,1,1,0,,1,0,0,0,0,0,0,0,0,,1,,3,4,0,0,0,0,0,1,0,1,79,79


In [8]:
preprocessed_df = data_unlabeled.drop(['Pas_ID', 'Ktr_andre', 'Oppdaget_annen', 'Sympt_annet', 'Verif_annet', 'Lok_annet', 'Hvorfor_ikke'], axis=1)

In [9]:
def mk_int(s):
    s = s.strip()
    
    try:
        s = int(s)
    except:
        s = float(s)
    finally:
        return s if s else None

In [10]:
def convert_to_int_float(data):
    header_names = list(data.columns.values)
    a = []
    for i in range(len(data.columns)):
        yes = (data.iloc[:,i].values).tolist()
        b = []
        for item in (yes):
            if(type(item)) == int or (type(item)) == float:
                b.append(item)
            else:
                b.append(mk_int(item))
        a.append(b)
    
    df = pd.DataFrame(a)
    df = df.transpose()
    df.columns = header_names
    return pd.DataFrame(df, columns=header_names)

In [11]:
preprocessed_df = convert_to_int_float(preprocessed_df)

In [12]:
preprocessed_df[:20]

,Født,Oppdaget_når,Age,Død,lever,FIGO_stadium,Substadium,histologi,gradering,kreftform,OVAgrade,myometireinvasjon,Cx_invasjon,Karinvasjon,Lymfekarinvasjon,Lymfeknuter,Tumor_str,hist_nummer,Primærbehandling,Avsluttet_prim_beh,Ant_ktr,Beh_sykehus,Ktr_ingen,Ktr_priv,Ktr_lok,Ktr_Uni,Ktr_komb,Ktr_fastlege,Lengde_sympt_dager,Lengde_sympt_uker,Lengde_sympt_mnd,Symptomer_mindre_1mnd,TidSympt_2dic,Oppdaget_pas,Oppdaget_FL,Oppdaget_Priv,Oppdaget_Lok,Oppdaget_Univ,Oppdaget_Kir,Tidligere_kontakt,Sympt_tilstede,Sympt_blødning,Sympt_smerter,Sympt_vekttap,Sympt_utflod,Sympt_ascites,Sympt_avføring,Sympt_ileus,Sympt_fatigue,Sympt_vannlating,Sympt_blodfeces,Sympt_blodurin,Sympt_anorexi,Sympt_kakeksi,Sympt_hudtumor,sympt_luftveier,Verif_histo,Histo_nummer,Histo_lab,Verif_cytol,Cytol_nummer,Cytol_lab,Verif_MR,Verif_CT,Verif_RTH,Verif_ABD_UL,Verif_VAG_UL,ULverif,Verif_Colon,Lok_vagina,Lok_bekken,Lok_ØvreAbd,Lok_lunger,Lok_lever,Lok_lok_LK,Lok_fjern_LK,Lok_skjelett,Lok_hjernen,Henvist_Uni,Behandling,Time_to_recurrence_months,symptomer_kliniske,KlinSympt_2dic,symptomer,Symtomer_tilstede,Sympt_tilstede2dic,Symptomer_lengde,Distal_recurrence,Local_recurrence,Age_at_diagnosis,age_rec
0,-1034.4708,1356.9948,75.0,NaN,NaN,1.0,3.0,1.0,2.0,1.0,NaN,NaN,2.0,2.0,NaN,2.0,NaN,NaN,1.0,1235.8620,8.0,5.0,NaN,0.0,0.0,5.0,NaN,NaN,5.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,4156.0,5.0,0.0,1.0,0.0,1.0,1.0,1.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,2.0,2.0,46.0,1.0,1.0,5.0,2.0,1.0,0.0,1.0,0.0,71.0,75.0
1,-909.7992,1448.9244,74.0,NaN,1.0,1.0,2.0,1.0,2.0,1.0,NaN,2.0,2.0,1.0,NaN,NaN,NaN,1503931.0,1.0,1425.1644,3.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,15160.0,1.0,1.0,16000088.0,NaN,0.0,1.0,0.0,1.0,1.0,1.0,NaN,1.0,0.0,1.0,NaN,0.0,0.0,1.0,0.0,0.0,1.0,3.0,9.0,3.0,1.0,1.0,2.0,1.0,0.0,1.0,0.0,73.0,74.0
2,-1126.0548,1406.8440,80.0,NaN,NaN,1.0,3.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,819216.0,5.0,1230.7644,16.0,2.0,NaN,1.0,0.0,0.0,NaN,NaN,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1427589.0,2.0,0.0,NaN,NaN,0.0,1.0,0.0,1.0,1.0,1.0,NaN,0.0,1.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,3.0,67.0,1.0,1.0,1.0,2.0,1.0,2.0,0.0,1.0,74.0,80.0
3,-1092.4452,1383.2604,78.0,NaN,NaN,1.0,1.0,6.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4884.0,1.0,1362.0924,2.0,5.0,NaN,0.0,0.0,5.0,NaN,NaN,0.0,3.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,632.0,5.0,0.0,NaN,NaN,0.0,1.0,0.0,1.0,1.0,1.0,NaN,0.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,3.0,8.0,1.0,1.0,3.0,2.0,1.0,0.0,1.0,0.0,77.0,78.0
4,-1069.1172,1438.3800,79.0,NaN,NaN,3.0,3.0,6.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23874.0,5.0,1427.8392,1.0,5.0,NaN,0.0,0.0,5.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,17694.0,5.0,0.0,NaN,NaN,0.0,1.0,0.0,1.0,1.0,1.0,NaN,1.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,79.0,79.0
5,-1189.7316,1446.3324,83.0,NaN,1.0,1.0,2.0,1.0,2.0,1.0,NaN,2.0,2.0,1.0,NaN,NaN,NaN,12628.0,1.0,1404.1656,6.0,11.0,NaN,1.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,1.0,0.0,0.0,0.0,1.0,1.0,NaN,0.0,1.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,NaN,16.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,82.0,83.0
6,-1210.2948,1391.2092,82.0,NaN,1.0,1.0,2.0,1.0,2.0,1.0,NaN,2.0,2.0,2.0,3.0,2.0,NaN,NaN,5.0,1320.1020,7.0,29.0,NaN,0.0,1.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1643.0,16.0,1.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,1.0,27.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,80.0,82.0
7,-1063.3284,1398.8952,78.0,NaN,NaN,1.0,2.0,1.0,1

In [13]:
unprocessable_data = preprocessed_df[['lever', 'Ktr_ingen', 'Ktr_fastlege', 'Tidligere_kontakt', 'Verif_Colon', 'Lok_ØvreAbd', 'Lok_lunger']]
tmp_columns = unprocessable_data.columns
imp_constant = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
tmp_arrays = imp_constant.fit_transform(unprocessable_data)
minor_dataset = pd.DataFrame(tmp_arrays, columns=tmp_columns)

In [14]:
preprocessed_df = preprocessed_df.drop(['lever', 'Ktr_ingen', 'Ktr_fastlege', 'Tidligere_kontakt', 'Verif_Colon', 'Lok_ØvreAbd', 'Lok_lunger'], axis=1)
header_names = preprocessed_df.columns

In [15]:
preprocessed_df[:20]

,Født,Oppdaget_når,Age,Død,FIGO_stadium,Substadium,histologi,gradering,kreftform,OVAgrade,myometireinvasjon,Cx_invasjon,Karinvasjon,Lymfekarinvasjon,Lymfeknuter,Tumor_str,hist_nummer,Primærbehandling,Avsluttet_prim_beh,Ant_ktr,Beh_sykehus,Ktr_priv,Ktr_lok,Ktr_Uni,Ktr_komb,Lengde_sympt_dager,Lengde_sympt_uker,Lengde_sympt_mnd,Symptomer_mindre_1mnd,TidSympt_2dic,Oppdaget_pas,Oppdaget_FL,Oppdaget_Priv,Oppdaget_Lok,Oppdaget_Univ,Oppdaget_Kir,Sympt_tilstede,Sympt_blødning,Sympt_smerter,Sympt_vekttap,Sympt_utflod,Sympt_ascites,Sympt_avføring,Sympt_ileus,Sympt_fatigue,Sympt_vannlating,Sympt_blodfeces,Sympt_blodurin,Sympt_anorexi,Sympt_kakeksi,Sympt_hudtumor,sympt_luftveier,Verif_histo,Histo_nummer,Histo_lab,Verif_cytol,Cytol_nummer,Cytol_lab,Verif_MR,Verif_CT,Verif_RTH,Verif_ABD_UL,Verif_VAG_UL,ULverif,Lok_vagina,Lok_bekken,Lok_lever,Lok_lok_LK,Lok_fjern_LK,Lok_skjelett,Lok_hjernen,Henvist_Uni,Behandling,Time_to_recurrence_months,symptomer_kliniske,KlinSympt_2dic,symptomer,Symtomer_tilstede,Sympt_tilstede2dic,Symptomer_lengde,Distal_recurrence,Local_recurrence,Age_at_diagnosis,age_rec
0,-1034.4708,1356.9948,75.0,NaN,1.0,3.0,1.0,2.0,1.0,NaN,NaN,2.0,2.0,NaN,2.0,NaN,NaN,1.0,1235.8620,8.0,5.0,0.0,0.0,5.0,NaN,5.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,4156.0,5.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,46.0,1.0,1.0,5.0,2.0,1.0,0.0,1.0,0.0,71.0,75.0
1,-909.7992,1448.9244,74.0,NaN,1.0,2.0,1.0,2.0,1.0,NaN,2.0,2.0,1.0,NaN,NaN,NaN,1503931.0,1.0,1425.1644,3.0,NaN,0.0,0.0,0.0,NaN,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,15160.0,1.0,1.0,16000088.0,NaN,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,9.0,3.0,1.0,1.0,2.0,1.0,0.0,1.0,0.0,73.0,74.0
2,-1126.0548,1406.8440,80.0,NaN,1.0,3.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,819216.0,5.0,1230.7644,16.0,2.0,1.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1427589.0,2.0,0.0,NaN,NaN,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,67.0,1.0,1.0,1.0,2.0,1.0,2.0,0.0,1.0,74.0,80.0
3,-1092.4452,1383.2604,78.0,NaN,1.0,1.0,6.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4884.0,1.0,1362.0924,2.0,5.0,0.0,0.0,5.0,NaN,0.0,3.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,632.0,5.0,0.0,NaN,NaN,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,8.0,1.0,1.0,3.0,2.0,1.0,0.0,1.0,0.0,77.0,78.0
4,-1069.1172,1438.3800,79.0,NaN,3.0,3.0,6.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23874.0,5.0,1427.8392,1.0,5.0,0.0,0.0,5.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,17694.0,5.0,0.0,NaN,NaN,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,79.0,79.0
5,-1189.7316,1446.3324,83.0,NaN,1.0,2.0,1.0,2.0,1.0,NaN,2.0,2.0,1.0,NaN,NaN,NaN,12628.0,1.0,1404.1656,6.0,11.0,1.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,16.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,82.0,83.0
6,-1210.2948,1391.2092,82.0,NaN,1.0,2.0,1.0,2.0,1.0,NaN,2.0,2.0,2.0,3.0,2.0,NaN,NaN,5.0,1320.1020,7.0,29.0,0.0,1.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1643.0,16.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,27.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,80.0,82.0
7,-1063.3284,1398.8952,78.0,NaN,1.0,2.0,1.0,1.0,1.0,NaN,2.0,2.0,2.0,3.0,2.0,NaN,1307185.0,1.0,1375.3080,3.0,1.0,0.0,1.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,12883.0,8.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,9.0

In [16]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
clean_data = imp_mean.fit_transform(preprocessed_df)

In [17]:
clean_df = pd.DataFrame(clean_data, columns=header_names)

In [18]:
clean_df[:20]

,Født,Oppdaget_når,Age,Død,FIGO_stadium,Substadium,histologi,gradering,kreftform,OVAgrade,myometireinvasjon,Cx_invasjon,Karinvasjon,Lymfekarinvasjon,Lymfeknuter,Tumor_str,hist_nummer,Primærbehandling,Avsluttet_prim_beh,Ant_ktr,Beh_sykehus,Ktr_priv,Ktr_lok,Ktr_Uni,Ktr_komb,Lengde_sympt_dager,Lengde_sympt_uker,Lengde_sympt_mnd,Symptomer_mindre_1mnd,TidSympt_2dic,Oppdaget_pas,Oppdaget_FL,Oppdaget_Priv,Oppdaget_Lok,Oppdaget_Univ,Oppdaget_Kir,Sympt_tilstede,Sympt_blødning,Sympt_smerter,Sympt_vekttap,Sympt_utflod,Sympt_ascites,Sympt_avføring,Sympt_ileus,Sympt_fatigue,Sympt_vannlating,Sympt_blodfeces,Sympt_blodurin,Sympt_anorexi,Sympt_kakeksi,Sympt_hudtumor,sympt_luftveier,Verif_histo,Histo_nummer,Histo_lab,Verif_cytol,Cytol_nummer,Cytol_lab,Verif_MR,Verif_CT,Verif_RTH,Verif_ABD_UL,Verif_VAG_UL,ULverif,Lok_vagina,Lok_bekken,Lok_lever,Lok_lok_LK,Lok_fjern_LK,Lok_skjelett,Lok_hjernen,Henvist_Uni,Behandling,Time_to_recurrence_months,symptomer_kliniske,KlinSympt_2dic,symptomer,Symtomer_tilstede,Sympt_tilstede2dic,Symptomer_lengde,Distal_recurrence,Local_recurrence,Age_at_diagnosis,age_rec
0,-1034.4708,1356.9948,75.0,1416.8268,1.0,3.0,1.0,2.0,1.0,3.0,2.0,2.0,2.0,3.0,2.0,5.0,1065523.0,1.0,1235.8620,8.0,5.0,0.0,0.0,5.0,1.0,5.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,170014.0,2.0,1.0,4156.0,5.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,46.0,1.0,1.0,5.0,2.0,1.0,0.0,1.0,0.0,71.0,75.0
1,-909.7992,1448.9244,74.0,1416.8268,1.0,2.0,1.0,2.0,1.0,3.0,2.0,2.0,1.0,3.0,2.0,5.0,1503931.0,1.0,1425.1644,3.0,5.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,15160.0,1.0,1.0,16000088.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,9.0,3.0,1.0,1.0,2.0,1.0,0.0,1.0,0.0,73.0,74.0
2,-1126.0548,1406.8440,80.0,1416.8268,1.0,3.0,1.0,1.0,1.0,3.0,2.0,2.0,2.0,3.0,2.0,5.0,819216.0,5.0,1230.7644,16.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1427589.0,2.0,0.0,18593.5,2.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,67.0,1.0,1.0,1.0,2.0,1.0,2.0,0.0,1.0,74.0,80.0
3,-1092.4452,1383.2604,78.0,1416.8268,1.0,1.0,6.0,3.0,1.0,3.0,2.0,2.0,2.0,3.0,2.0,5.0,4884.0,1.0,1362.0924,2.0,5.0,0.0,0.0,5.0,1.0,0.0,3.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,632.0,5.0,0.0,18593.5,2.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,8.0,1.0,1.0,3.0,2.0,1.0,0.0,1.0,0.0,77.0,78.0
4,-1069.1172,1438.3800,79.0,1416.8268,3.0,3.0,6.0,3.0,1.0,3.0,2.0,2.0,2.0,3.0,2.0,5.0,23874.0,5.0,1427.8392,1.0,5.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,17694.0,5.0,0.0,18593.5,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,79.0,79.0
5,-1189.7316,1446.3324,83.0,1416.8268,1.0,2.0,1.0,2.0,1.0,3.0,2.0,2.0,1.0,3.0,2.0,5.0,12628.0,1.0,1404.1656,6.0,11.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,170014.0,2.0,0.0,18593.5,2.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,16.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,82.0,83.0
6,-1210.2948,1391.2092,82.0,1416.8268,1.0,2.0,1.0,2.0,1.0,3.0,2.0,2.0,2.0,3.0,2.0,5.0,1065523.0,5.0,1320.1020,7.0,29.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,170014.0,2.0,1.0,1643.0,16.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,27.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,80.0,82.0
7,-1063.3284,1398.8952,78.0,1416.8268,1.0,2.0,1.0,1.0,1.0,3.0,2.0,2.0,2.0,3.0,2.0,5.0,1307185.0,1.0,1375.3080,3.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [19]:
processed_df = pd.concat([clean_df, minor_dataset], axis=1)

In [20]:
processed_df.head()

,Født,Oppdaget_når,Age,Død,FIGO_stadium,Substadium,histologi,gradering,kreftform,OVAgrade,myometireinvasjon,Cx_invasjon,Karinvasjon,Lymfekarinvasjon,Lymfeknuter,Tumor_str,hist_nummer,Primærbehandling,Avsluttet_prim_beh,Ant_ktr,Beh_sykehus,Ktr_priv,Ktr_lok,Ktr_Uni,Ktr_komb,Lengde_sympt_dager,Lengde_sympt_uker,Lengde_sympt_mnd,Symptomer_mindre_1mnd,TidSympt_2dic,Oppdaget_pas,Oppdaget_FL,Oppdaget_Priv,Oppdaget_Lok,Oppdaget_Univ,Oppdaget_Kir,Sympt_tilstede,Sympt_blødning,Sympt_smerter,Sympt_vekttap,Sympt_utflod,Sympt_ascites,Sympt_avføring,Sympt_ileus,Sympt_fatigue,Sympt_vannlating,Sympt_blodfeces,Sympt_blodurin,Sympt_anorexi,Sympt_kakeksi,Sympt_hudtumor,sympt_luftveier,Verif_histo,Histo_nummer,Histo_lab,Verif_cytol,Cytol_nummer,Cytol_lab,Verif_MR,Verif_CT,Verif_RTH,Verif_ABD_UL,Verif_VAG_UL,ULverif,Lok_vagina,Lok_bekken,Lok_lever,Lok_lok_LK,Lok_fjern_LK,Lok_skjelett,Lok_hjernen,Henvist_Uni,Behandling,Time_to_recurrence_months,symptomer_kliniske,KlinSympt_2dic,symptomer,Symtomer_tilstede,Sympt_tilstede2dic,Symptomer_lengde,Distal_recurrence,Local_recurrence,Age_at_diagnosis,age_rec,lever,Ktr_ingen,Ktr_fastlege,Tidligere_kontakt,Verif_Colon,Lok_ØvreAbd,Lok_lunger
0,-1034.4708,1356.9948,75.0,1416.8268,1.0,3.0,1.0,2.0,1.0,3.0,2.0,2.0,2.0,3.0,2.0,5.0,1065523.0,1.0,1235.8620,8.0,5.0,0.0,0.0,5.0,1.0,5.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,170014.0,2.0,1.0,4156.0,5.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,46.0,1.0,1.0,5.0,2.0,1.0,0.0,1.0,0.0,71.0,75.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-909.7992,1448.9244,74.0,1416.8268,1.0,2.0,1.0,2.0,1.0,3.0,2.0,2.0,1.0,3.0,2.0,5.0,1503931.0,1.0,1425.1644,3.0,5.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,15160.0,1.0,1.0,16000088.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,9.0,3.0,1.0,1.0,2.0,1.0,0.0,1.0,0.0,73.0,74.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,-1126.0548,1406.8440,80.0,1416.8268,1.0,3.0,1.0,1.0,1.0,3.0,2.0,2.0,2.0,3.0,2.0,5.0,819216.0,5.0,1230.7644,16.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1427589.0,2.0,0.0,18593.5,2.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,67.0,1.0,1.0,1.0,2.0,1.0,2.0,0.0,1.0,74.0,80.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,-1092.4452,1383.2604,78.0,1416.8268,1.0,1.0,6.0,3.0,1.0,3.0,2.0,2.0,2.0,3.0,2.0,5.0,4884.0,1.0,1362.0924,2.0,5.0,0.0,0.0,5.0,1.0,0.0,3.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,632.0,5.0,0.0,18593.5,2.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,8.0,1.0,1.0,3.0,2.0,1.0,0.0,1.0,0.0,77.0,78.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,-1069.1172,1438.3800,79.0,1416.8268,3.0,3.0,6.0,3.0,1.0,3.0,2.0,2.0,2.0,3.0,2.0,5.0,23874.0,5.0,1427.8392,1.0,5.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,17694.0,5.0,0.0,18593.5,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,79.0,79.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
header_names = processed_df.columns

In [22]:
from scipy import stats
normalized_df = stats.zscore(processed_df)

In [23]:
normalized_df = pd.DataFrame(normalized_df, columns=header_names)

In [24]:
normalized_df.head()

,Født,Oppdaget_når,Age,Død,FIGO_stadium,Substadium,histologi,gradering,kreftform,OVAgrade,myometireinvasjon,Cx_invasjon,Karinvasjon,Lymfekarinvasjon,Lymfeknuter,Tumor_str,hist_nummer,Primærbehandling,Avsluttet_prim_beh,Ant_ktr,Beh_sykehus,Ktr_priv,Ktr_lok,Ktr_Uni,Ktr_komb,Lengde_sympt_dager,Lengde_sympt_uker,Lengde_sympt_mnd,Symptomer_mindre_1mnd,TidSympt_2dic,Oppdaget_pas,Oppdaget_FL,Oppdaget_Priv,Oppdaget_Lok,Oppdaget_Univ,Oppdaget_Kir,Sympt_tilstede,Sympt_blødning,Sympt_smerter,Sympt_vekttap,Sympt_utflod,Sympt_ascites,Sympt_avføring,Sympt_ileus,Sympt_fatigue,Sympt_vannlating,Sympt_blodfeces,Sympt_blodurin,Sympt_anorexi,Sympt_kakeksi,Sympt_hudtumor,sympt_luftveier,Verif_histo,Histo_nummer,Histo_lab,Verif_cytol,Cytol_nummer,Cytol_lab,Verif_MR,Verif_CT,Verif_RTH,Verif_ABD_UL,Verif_VAG_UL,ULverif,Lok_vagina,Lok_bekken,Lok_lever,Lok_lok_LK,Lok_fjern_LK,Lok_skjelett,Lok_hjernen,Henvist_Uni,Behandling,Time_to_recurrence_months,symptomer_kliniske,KlinSympt_2dic,symptomer,Symtomer_tilstede,Sympt_tilstede2dic,Symptomer_lengde,Distal_recurrence,Local_recurrence,Age_at_diagnosis,age_rec,lever,Ktr_ingen,Ktr_fastlege,Tidligere_kontakt,Verif_Colon,Lok_ØvreAbd,Lok_lunger
0,-0.942533,-0.823484,0.835434,0.049607,-1.405504,0.674497,-0.715975,-0.926601,-1.311455,0.193329,0.253773,0.127869,0.117936,0.136631,0.074658,0.025582,-0.079235,-1.616521,-0.755951,0.533437,-0.337448,-0.338754,-0.891176,2.732502,-0.187103,2.785543,-0.389041,-0.376591,1.937845,1.035940,1.422049,-0.269361,-0.25,-0.745697,-0.566029,-0.174078,0.918051,-0.313920,1.441153,-0.218554,-0.115814,-0.29074,-0.296648,-0.144986,-0.311086,-0.24329,-0.066568,-0.115814,-0.207231,-0.054313,-0.195366,-0.178512,-0.940030,-0.087074,-0.308196,1.461666,-0.115925,1.282632,-0.338754,0.453513,-0.160128,4.654747,2.251983,2.047154,-0.529719,-0.794163,-0.19435,-0.460566,2.288784,-0.13403,-0.101986,2.391921,-0.887958,0.501161,0.128841,0.956814,1.356037,1.104020,0.824022,-0.703791,0.670741,-0.670741,0.658558,0.835434,-0.518321,-0.054313,-0.178512,1.362770,-0.101986,-0.750479,-0.439298
1,-0.593181,1.611260,0.746475,0.049607,-1.405504,-0.578666,-0.715975,-0.926601,-1.311455,0.193329,0.253773,0.127869,-3.089932,0.136631,0.074658,0.025582,-0.060010,-1.616521,0.985784,-0.539115,-0.337448,-0.338754,-0.891176,-0.577262,-0.187103,-0.197401,0.914150,-0.376591,1.937845,1.035940,1.422049,-0.269361,-0.25,1.341027,1.766695,-0.174078,0.918051,3.185521,1.441153,-0.218554,-0.115814,-0.29074,-0.296648,-0.144986,3.214550,-0.24329,-0.066568,-0.115814,-0.207231,-0.054313,-0.195366,-0.178512,1.063796,-0.116975,-0.675869,1.461666,2.449880,-0.190661,-0.338754,0.453513,-0.160128,4.654747,2.251983,2.047154,1.887792,-0.794163,-0.19435,-0.460566,2.288784,-0.13403,-0.101986,-0.079973,-0.149802,-0.460054,1.954083,0.956814,-0.169995,1.104020,0.824022,-0.703791,0.670741,-0.670741,0.831195,0.746475,1.929306,-0.054313,-0.178512,1.362770,-0.101986,1.332483,-0.439298
2,-1.199168,0.496766,1.280232,0.049607,-1.405504,0.674497,-0.715975,-2.220418,-1.311455,0.193329,0.253773,0.127869,0.117936,0.136631,0.074658,0.025582,-0.090036,0.550525,-0.802853,2.249520,-0.686178,2.951997,-0.891176,-0.577262,-0.187103,-0.197401,-0.389041,0.084818,-0.516037,1.035940,1.422049,-0.269361,4.00,-0.745697,1.766695,-0.174078,0.918051,-0.313920,1.441153,-0.218554,-0.115814,-0.29074,-0.296648,-0.144986,-0.311086,-0.24329,-0.066568,-0.115814,-0.207231,-0.054313,-0.195366,-0.178512,1.063796,0.155759,-0.308196,-0.329204,-0.113609,-0.190661,-0.338754,0.453513,-0.160128,4.654747,2.251983,2.047154,-0.529719,1.259187,-0.19435,-0.460566,-0.436913,-0.13403,-0.101986,-0.079973,-0.149802,1.046716,0.128841,0.956814,-0.169995,1.104020,0.824022,0.172725,-1.490889,1.490889,0.917514,1.280232,-0.518321,-0.054313,-0.178512,1.362770,-0.101986,-0.750479,-0.439298
3,-1.104988,-0.127843,1.102313,0.049607,-1.405504,-1.831829,1.546400,0.367216,-1.311455,0.193329,0.253773,0.127869,0.117936,0.136631,0.074658,0.025582,-0.125746,-1.616521,0.405471,-0.753626,-0.337448,-0.338754,-0.891176,2.73

In [25]:
%store normalized_df

Stored 'normalized_df' (DataFrame)
